In [17]:
import pandas as pd
from surprise.model_selection import train_test_split
from surprise import Dataset, Reader, SVD, accuracy, AlgoBase 
import numpy as np

In [18]:
all_books = pd.read_csv('data/Books.csv')
all_ratings = pd.read_csv('data/Ratings.csv')
all_users = pd.read_csv('data/Users.csv')
all_books.head()

C:\Users\Ethan Rody\AppData\Local\Temp\ipykernel_27260\2654531577.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  all_books = pd.read_csv('data/Books.csv')


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [19]:
# Checks if item is a valid ISBN
def is_valid_isbn(isbn):
     if len(isbn) != 10: return False
     if not isbn.isalnum(): return False

     return True

# Converts ISBN to numerical ID
def convert_isbn_to_id(isbn, isbn_to_id):
    return isbn_to_id.get(isbn, None)

data_list = []
isbn_to_id = {}
id_counter = 0

# Converts all ISBNs in data to numerical IDs
with open('data/Ratings.csv', 'r') as file:
    next(file)
    for line in file:
        # Skips misformatted items that would cause an error
        try:
            item_id, isbn, rating = line.strip().split(',')
        except:
            continue
        
        # Skips misformatted items (not valid ISBNs)
        if not is_valid_isbn(isbn):
            continue

        # Converts ISBN to numerical ID
        if isbn not in isbn_to_id:
            isbn_to_id[isbn] = id_counter
            id_counter += 1
        
        # Builds dictionary
        item_id = isbn_to_id[isbn]

        # Add data to list of tuples
        data_list.append((item_id, item_id, float(rating)))

# Creates inverted dictionary, use to convert numerical ID to ISBN for reporting results
id_to_isbn = {v: k for k, v in isbn_to_id.items()}

# Converts list to pandas data frame
df = pd.DataFrame(data_list, columns=['user_id', 'item_id', 'rating'])

# Creates Reader to extract data, uses it to load data
reader = Reader(line_format="user item rating", sep=',', rating_scale=(1,10))
data = Dataset.load_from_df(df, reader=reader)

# Splits the data into train_data and test_data
train_data, test_data = train_test_split(data, test_size=0.8)

In [20]:
# Custom weighted sum recommendation system algorithm
class WeightedSum(AlgoBase):
    # Initializes algorithm using a list of models to serve as base
    def __init__(self, models, learn_rate=0.01, threshold=1e-5, perturbation=1e-5):
        AlgoBase.__init__(self)

        self.models = models
        self.weights = [1/len(models)]*len(models)
        self.learn_rate = learn_rate
        self.threshold = threshold
        self.perturbation = perturbation
    
    # Fits algorithm to data, optimizes weights
    def fit(self, trainset):
        AlgoBase.fit(self, trainset)
        
        # Splits trainset into train_data and optimize_data
        temp_df = pd.DataFrame(trainset.all_ratings(), columns=['user_id', 'item_id', 'rating'])
        temp_data = Dataset.load_from_df(temp_df, reader=reader)
        train_data, optimize_data = train_test_split(temp_data, test_size=0.4)

        # Fits base models to train_data
        for model in self.models:
            model.fit(train_data)
        
        # Optimizes weights using optimize_data through gradient descent
        change = 1

        pred = self.test(optimize_data)
        acc = accuracy.rmse(pred, False)
        prev_acc = acc
        while change > self.threshold:
            # Copies current state of weights
            curr_weights = self.weights.copy()
            
            # Computes gradients
            gradients = []
            for i in range(len(self.weights)):
                weight = self.weights[i]

                self.weights[i] += self.perturbation
                new_pred = self.test(optimize_data)
                new_acc = accuracy.rmse(new_pred, False)

                self.weights[i] = weight
                pred = self.test(optimize_data)
                acc = accuracy.rmse(pred, False)

                gradient = (new_acc - acc) / self.perturbation
                gradients.append(gradient)

            # Restores state of weights
            self.weights = curr_weights

            # Modifies weights
            for i in range(len(self.weights)):
                self.weights[i] -= self.learn_rate * gradients[i]
                # Minimum value for weight is 0
                self.weights[i] = max(self.weights[i], 0)

            # Ensures weights sum to 1
            for i in range(len(self.weights)):
                self.weights[i] = self.weights[i] / sum(self.weights)

            # Computes new accuracy
            pred = self.test(optimize_data)
            acc = accuracy.rmse(pred, False)
            change = abs(acc - prev_acc)
            prev_acc = acc
        
        return self

    # Estimates rating for given user and item
    def estimate(self, u , i):
        predictions = []
        for model in self.models:
            predictions.append(model.predict(u, i))

        final_prediction = 0
        for i in range(0, len(self.weights)):
            final_prediction += predictions[i].est*self.weights[i]

        return final_prediction

In [21]:
# Custom Recommendation system algorithm that estimates rating as global mean
class GlobalMean(AlgoBase):
    def __init__(self):
        AlgoBase.__init__(self)

    def fit(self, trainset):
        AlgoBase.fit(self, trainset)

        self.mean_rating = trainset.global_mean

        return self

    def estimate(self, u, i):

        return self.mean_rating

# Custom Recommendation system algorithm that estimates rating as mean rating of the user
class UserMean(AlgoBase):
    def __init__(self):
        AlgoBase.__init__(self)

    def fit(self, trainset):
        AlgoBase.fit(self, trainset)

        self.means = {}
        for user_id in trainset.all_users():
            user_ratings = [0]
            for rating in trainset.ur[user_id]:
                _, score = rating
                if score != 0:
                    user_ratings.append(score)

            # If user has no ratings: estimate is global mean
            if len(user_ratings) == 1:
                user_mean = trainset.global_mean
            else:
                user_mean = sum(user_ratings) / (len(user_ratings)-1)         

            self.means[user_id] = user_mean

        return self

    def estimate(self, u, i):
        return self.means.get(u, self.trainset.global_mean)

# Custom Recommendation system algorithm that estimates rating as mean rating of the item
class ItemMean(AlgoBase):
    def __init__(self):
        AlgoBase.__init__(self)

    def fit(self, trainset):
        AlgoBase.fit(self, trainset)

        self.means = {}
        for item_id in trainset.all_items():
            item_ratings = [0]
            for rating in trainset.ir[item_id]:
                _, score = rating
                if score != 0:
                    item_ratings.append(score)

            # If item has no ratings: estimate is global mean
            if len(item_ratings) == 1:
                item_mean = trainset.global_mean
            else:
                item_mean = sum(item_ratings) / (len(item_ratings)-1)         

            self.means[item_id] = item_mean

        return self

    def estimate(self, u, i):
        return self.means.get(i, self.trainset.global_mean)

In [22]:
gm = GlobalMean()
um = UserMean()
im = ItemMean()
svd = SVD()

gm.fit(train_data)
um.fit(train_data)
im.fit(train_data)
svd.fit(train_data)

gm_pred = gm.test(test_data)
um_pred = um.test(test_data)
im_pred = im.test(test_data)
svd_pred = svd.test(test_data)

print("Global Mean Model Accuracy: {:.5f}".format(accuracy.rmse(gm_pred, False)))
print("User Mean Model Accuracy: {:.5f}".format(accuracy.rmse(um_pred, False)))
print("Item Mean Model Accuracy: {:.5f}".format(accuracy.rmse(im_pred, False)))
print("SVD Model Accuracy: {:.5f}".format(accuracy.rmse(svd_pred, False)))

Global Mean Model Accuracy: 3.85268
User Mean Model Accuracy: 5.04057
Item Mean Model Accuracy: 5.04057
SVD Model Accuracy: 3.91872


In [23]:
# Declares models for use in weighted sum (NOTE: this takes ~145min)
svd = SVD()
gm = GlobalMean()
um = UserMean()
im = ItemMean()

models = [svd, gm, um, im]

# Declares weighted sum, fits to train_data
weighted_sum = WeightedSum(models)
weighted_sum.fit(train_data)

In [24]:
# Reports accuracy
predictions = weighted_sum.test(test_data)
accuracy.rmse(predictions)

print("Optimal Weights: ", weighted_sum.weights)

RMSE: 3.8585
Optimal Weights:  [0.5725664389874676, 0.4268179940139243, 0.0, 0.0]


In [25]:
#converts prediction vector to np.array
arr = np.array(predictions)
pred_arr = np.array([i[:4] for i in arr])
#gets the values user id, item id, user's rating of item, and prediction score (can be expanded for more data)
df = pd.DataFrame(pred_arr, columns=['uid','iid','rating','score'])
#drop duplicate ISBN to prevent repeat predictions (currently most likely drops duplicate ISBNs without a rating first)
df = df.drop_duplicates('iid')
df["ISBN"] = df['iid'].apply(lambda x: id_to_isbn[x])
df = df.sort_values("score", ascending = False)
for isbn in df['ISBN'][:5]:
    recc_book = all_books.loc[all_books["ISBN"] == isbn]['Book-Title'].to_string(index = False)
    print(recc_book)

Free
Die Gefahrten I
Wild
2001: A Space Odyssey
Dogs and Their Women
